In [1]:
import csv 
import numpy as np
from numpy.linalg import inv #矩阵求逆
import random
import math
import sys
import pandas as pd

In [2]:
file = open('data/train.csv', 'r',encoding='big5')
read_data = csv.reader(file,delimiter=',')

data = []
for i in range(18):
    data.append([])
idx = 0
for row in read_data:
    if idx!=0:
        for i in range(3,27):
            if row[i]!='NR':
                data[(idx-1)%18].append(float(row[i]))
            else:
                data[(idx-1)%18].append(float(0))
    idx+=1
file.close()

In [14]:
file = open('data/test.csv', 'r',encoding='big5')
read_data = csv.reader(file, delimiter=',')
idx = 0
test_data = []
for row in read_data:
    